In [23]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:13 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Fetched 88.7 kB in 2s (36.2 kB/s)
Reading package lists... Done


In [24]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").getOrCreate()

In [25]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [26]:
# filter DataFrame for total_votes above or equal to 20
df_total_vote = df.filter(df.total_votes >= 20)
df_total_vote.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48701722| RBEMQ29WJBHYG|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|          5|          164|        168|   N|                Y|  I LOVE my recorder|I LOVE my recorde...| 2015-08-31|
|         US|   40963441|R2JK5Y8D5MXAGP|B00SU7B0JK|      82850235|iXCC Multi pack L...|Mobile_Electronics|      

In [27]:
# Filter DataFrame for helpful_votes ratio above or equal to 50%
df_filter_half = df_total_vote.filter((df_total_vote.helpful_votes/df_total_vote.total_votes) >= 0.5)
df_filter_half.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48701722| RBEMQ29WJBHYG|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|          5|          164|        168|   N|                Y|  I LOVE my recorder|I LOVE my recorde...| 2015-08-31|
|         US|   25750755|R13W3EMIUV120L|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|      

In [28]:
# Create paid vine DataFrame
paid_df = df_filter_half.filter(df_filter_half.vine == "Y")
paid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52490988|R3BOQTLUU3Y4L8|B00EP0SIOG|     628459233|Orbotix Sphero 2....|Mobile_Electronics|          4|           42|         55|   Y|                N|Lot of fun, but f...|Before receiving ...| 2013-11-19|
|         US|   48814959| RWF03LXVXC22A|B00EP0SIOG|     628459233|Orbotix Sphero 2....|Mobile_Electronics|      

In [29]:
unpaid_df = df_filter_half.filter(df_filter_half.vine == "N")
unpaid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48701722| RBEMQ29WJBHYG|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|          5|          164|        168|   N|                Y|  I LOVE my recorder|I LOVE my recorde...| 2015-08-31|
|         US|   25750755|R13W3EMIUV120L|B00YO3UYXW|     913911909|Jensen MCR-100 Ca...|Mobile_Electronics|      

In [30]:
# paid total number of reviews
total_paid_reviews = paid_df.count()
total_paid_reviews

4

In [31]:
# paid 5-star reviews
total_fivestar_reviews = paid_df.filter(paid_df.star_rating == 5).count()
total_fivestar_reviews

1

In [32]:
total_fourstar_reviews = paid_df.filter(paid_df.star_rating == 4).count()
total_fourstar_reviews

2

In [33]:
total_threestar_reviews = paid_df.filter(paid_df.star_rating == 3).count()
total_threestar_reviews

1

In [34]:
total_twostar_reviews = paid_df.filter(paid_df.star_rating == 2).count()
total_twostar_reviews

0

In [35]:
total_onestar_reviews = paid_df.filter(paid_df.star_rating == 1).count()
total_onestar_reviews

0

In [36]:
# paid 5-star reviews percentage

paid_fivestar_percent = (total_fivestar_reviews / total_paid_reviews) * 100
paid_fivestar_percent

25.0

In [37]:
paid_fourstar_percent = (total_fourstar_reviews / total_paid_reviews) * 100
paid_fourstar_percent

50.0

In [38]:
paid_threestar_percent = (total_threestar_reviews / total_paid_reviews) * 100
paid_threestar_percent

25.0

In [39]:
paid_twostar_percent = (total_twostar_reviews / total_paid_reviews) * 100
paid_twostar_percent

0.0

In [40]:
paid_onestar_percent = (total_onestar_reviews / total_paid_reviews) * 100
paid_onestar_percent

0.0

In [41]:
# unpaid total number of reviews
total_unpaid_reviews = unpaid_df.count()
total_unpaid_reviews

1064

In [42]:
unpaid_fivestar_review = unpaid_df.filter(paid_df.star_rating == 5).count()
unpaid_fivestar_review

527

In [43]:
unpaid_fourstar_review = unpaid_df.filter(paid_df.star_rating == 4).count()
unpaid_fourstar_review

182

In [44]:
unpaid_threestar_review = unpaid_df.filter(paid_df.star_rating == 3).count()
unpaid_threestar_review

95

In [45]:
unpaid_twostar_review = unpaid_df.filter(paid_df.star_rating == 2).count()
unpaid_twostar_review

63

In [46]:
unpaid_onestar_review = unpaid_df.filter(paid_df.star_rating == 1).count()
unpaid_onestar_review

197

In [47]:
unpaid_fivestar_percent = (unpaid_fivestar_review / total_unpaid_reviews) * 100
unpaid_fivestar_percent


49.53007518796993

In [48]:
unpaid_fourstar_percent = (unpaid_fourstar_review / total_unpaid_reviews) * 100
unpaid_fourstar_percent

17.105263157894736

In [49]:
unpaid_threestar_percent = (unpaid_threestar_review / total_unpaid_reviews) * 100
unpaid_threestar_percent

8.928571428571429

In [50]:
unpaid_twostar_percent = (unpaid_twostar_review / total_unpaid_reviews) * 100
unpaid_twostar_percent

5.921052631578947

In [53]:
unpaid_onestar_percent = (unpaid_onestar_review / total_unpaid_reviews) * 100
print("unpaid_onestar_percent", unpaid_onestar_percent)

unpaid_onestar_percent 18.515037593984964


In [56]:
#Unpaid Results
print("Unpaid Results")
print("unpaid_fivestar_percent", unpaid_fivestar_percent)
print("unpaid_fourstar_percent", unpaid_fourstar_percent)
print("unpaid_threestar_percent", unpaid_threestar_percent)
print("unpaid_twostar_percent", unpaid_twostar_percent)
print("unpaid_onestar_percent", unpaid_onestar_percent)

Unpaid Results
unpaid_fivestar_percent 49.53007518796993
unpaid_fourstar_percent 17.105263157894736
unpaid_threestar_percent 8.928571428571429
unpaid_twostar_percent 5.921052631578947
unpaid_onestar_percent 18.515037593984964


In [57]:
#Paid Results
print("paid Results")
print("paid_fivestar_percent", paid_fivestar_percent)
print("paid_fourstar_percent", paid_fourstar_percent)
print("paid_threestar_percent", paid_threestar_percent)
print("paid_twostar_percent", paid_twostar_percent)
print("paid_onestar_percent", paid_onestar_percent)

paid Results
paid_fivestar_percent 25.0
paid_fourstar_percent 50.0
paid_threestar_percent 25.0
paid_twostar_percent 0.0
paid_onestar_percent 0.0
